## Transformation

In [ ]:
%pyspark 

data = [1,2,3,4,5]
rdd = sc.parallelize(data, 4) 
rdd.collect()

In [ ]:
%pyspark

lines = sc.textFile('file:/tmp/trump.txt') 
lines.take(3)

In [ ]:
%pyspark
def timesTwo(e):
    return e * 2
    
rdd = sc.parallelize([1, 2, 3, 4]) 
rdd2 = rdd.map(timesTwo)
rdd2.collect()

In [ ]:
%pyspark

rdd = sc.parallelize([1, 2, 3, 4]) 
rdd2 = rdd.map(lambda e: e * 2)
rdd2.collect()

In [ ]:
%pyspark
rdd2 = rdd.filter(lambda x: x % 2 == 0)
rdd2.collect()

In [ ]:
%pyspark
rdd = sc.parallelize([1,4,2,2,3])
rdd2 = rdd.distinct()
rdd2.collect()

In [ ]:
%pyspark
rdd=sc.parallelize([1,2,3])
rdd2 = rdd.map(lambda x:[x,x+5])
rdd2.collect()

In [ ]:
%pyspark
rdd2 = rdd.flatMap(lambda x:[x,x+5])
rdd2.collect()

## Action

In [ ]:
rdd=sc.parallelize([1,2,3])
rdd.reduce(lambda x,y: x + y)

In [ ]:
rdd.take(2)

In [ ]:
rdd.collect()

In [ ]:
rdd=sc.parallelize([5,3,1,2]) 
rdd.takeOrdered(3,lambda s:-1*s)

## Key-Value RDD

In [ ]:
rdd  = sc.parallelize([(1,2), (3,4), (3,6)]) 
rdd2 = rdd.reduceByKey(lambda a, b: a + b)
rdd2.collect()

In [ ]:
rdd2 = sc.parallelize([(1,'a'), (2,'c'), (1,'b')]) 
rdd3 = rdd2.sortByKey()
rdd3.collect()

In [ ]:
rdd2 = sc.parallelize([(1,'a'), (2,'c'), (1,'b')]) 
rdd3 = rdd2.groupByKey()
rdd3.collect()

## Broadcast, Accumulator

In [ ]:
%pyspark
broadcastVar = sc.broadcast([1, 2, 3])
broadcastVar.value

In [ ]:
%pyspark
accum = sc.accumulator(0) 
rdd = sc.parallelize([1, 2, 3, 4])

def f(x):
    global accum 
    accum += x
    
rdd.foreach(f) 

accum.value